In [ ]:
import requests
import json
import time
import pandas as pd
from IPython.display import Markdown  

In [ ]:
import requests
import json
from IPython.display import Markdown

url = "https://chatai.1rmb.tk/api/v1/chat/completions"
#url = "https://closeai.deno.dev/v1/chat/completions"
#url = "https://api.lixining.com/v1/chat/completions"
#返向服务器地址 如果失效了可以更换

api_key = ''#你的API

headers = {
  'Authorization': f'Bearer {api_key}',
  'Content-Type': 'application/json'
}

In [ ]:
#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

In [ ]:
get_completion("hello world")

'Hello! How can I assist you today?'

In [ ]:
text=f"""std::vector<int> v = {2, 15, 1, 8, 3};"""
prompt = f"""
我希望你能充当代码解释者，帮我阐明下面三个引号括起来的代码的语法和语义。使用markdown格式回复我。\。
\"\"\"{text}\"\"\"
"""
response = get_completion(prompt)
Markdown(response)

语法：

这行代码定义了一个名为v的std::vector<int>类型的变量，并给它赋初值。初始值是通过一个包含5个元素的括号表达式来实现的，这个表达式使用逗号分隔每个值。

语义：

该代码定义了一个std::vector<int>类型的变量v，并初始化为{2, 15, 1, 8, 3}，其中逗号用于分隔每个值。这里使用逗号运算符的目的是对于一个由不同值组成的序列，将它们组合在一起，从而生成一个括号表达式的值为最后一个元素的序列。因此，(2, 15, 1, 8, 3)表示一个值为3的序列，并将其分配给v。

In [ ]:
def chatgpt(text):
    prompt = f"""
    我希望你能作为一个诗人。你要创作出能唤起人们情感并有力量搅动人们灵魂的诗篇。\
    写任何话题或主题，但要确保你的文字以美丽而有意义的方式传达你所要表达的感觉。你也可以想出一些短小的诗句，\
    但仍有足够的力量在读者心中留下印记。请用下面三个引号括起来的文本，创作一首古诗，使用markdown格式回复我。\。
    \"\"\"{text}\"\"\"
    """
    response = get_completion(prompt)
    return Markdown(response)

In [ ]:
chatgpt("思绪随风逝，往事转眼休。")

In [ ]:
def title(text):
    text_1 = f"""
    {text}
    """
    prompt = f"""
    您将获得由三个引号括起来的文本。\
    请在这段里面找某个角度，拟定一个调研课题，并进行简单的设计，使用markdown格式回复我。


    \"\"\"{text_1}\"\"\"
    """
    response = get_completion(prompt)
    #print("调研题目:")
    return (response)

In [ ]:
def UsereMotions(text):
    prompt = f"""
    下面三个引号括起来的内容是某部影视作品的短评，帮我根据内容的感情色彩分为正面、负面评价，分类好后，直接选择一个最接近的结果答复我，不要回复其他内容。\。
    \"\"\"{text}\"\"\"
    """
    response = get_completion(prompt)
    return (response)

In [ ]:
评论内容=f"""
国产主题剧里只要是出现了为煽情而煽情的画面一般都有点……一个牛逼哄哄自信\
爆棚的年轻人总是有点桀骜不驯然后被老选手九死一生拯救回来，\
把自信心揉碎再乖乖学习的老套路一成不变，在多少电影里看过了。
"""
UsereMotions(评论内容)

'负面评价。'

In [ ]:
import pandas as pd

# 评论列表
string_list = ["没有耳机，还得自己花钱再买耳机", "外形外观：颜值不错，手感也很好", "用着挺好的，还挺满意","大字体，大图标，细心教老人很喜欢用","卡顿很严重，发热控制不好"]

df = pd.DataFrame({'评价内容': string_list, '评价分类': None})
df

In [ ]:
df['评价分类'] = df['评价内容'].apply(UsereMotions)
df

In [ ]:
import os 
import ast

def read_list_from_env_file(file_path):# 从文件中读取列表
    with open(file_path) as f:
        content = f.read().strip()
    # 从字符串中解析出列表
    lst = ast.literal_eval(content)
    return lst

In [ ]:
api_keys=read_list_from_env_file(".env")# 读取.env文件中的API_KEY

In [ ]:
class APIPool:
    def __init__(self,tokens):
        self.tokens=tokens
        self.last_called={token:0 for token in tokens}  #上次调用时间
    def getapi(self):
        now = int(time.time())
        for token in self.tokens:
            if self.last_called[token]+21 <=now:    
                self.last_called[token]=now
                return token
        time.sleep(self.last_called[self.tokens[0]]+21-now) #等待
        return self.getapi() #重新调用
    def reset(self):
        self.last_called={token:0 for token in self.tokens}#重置

In [ ]:
print("hello world!")

In [ ]:
A=APIPool(api_keys) #创建API池

In [ ]:
url = "https://api.lixining.com/v1/chat/completions" # 选择一个API
#反向代理 如失效需要 更换到上面
#https://openai.1rmb.tk/v1/chat/completions
#https://ai.fakeopen.com/v1/chat/completions
#https://api.lixining.com/v1/chat/completions

In [ ]:
def get_header():# 获取请求头
    header={
  'Authorization': f'Bearer {A.getapi()}',
  'Content-Type': 'application/json'
    }
    return header

In [ ]:
print(get_header())

In [ ]:
#获取消息的函数
def get_completion(prompt):
    payload = {
      "model": "gpt-3.5-turbo",
      "messages": [
          {"role": "system", "content": f"You are ChatGPT, a large language model trained by OpenAI. \
            Follow the user's instructions carefully. Respond using markdown."},
        {
          "role": "user",
          "content": f"{prompt}"
        }
      ]
    }
    data=None
    try:
        response = requests.post(url, headers=get_header(), json=payload)
        response.raise_for_status() # 抛出异常，如果响应码不是200
        data = response.json()
        return (data["choices"][0]["message"]["content"])
    except requests.exceptions.RequestException as e:
        print(f"请求错误: {e} \n {data}")
    except json.JSONDecodeError as e:
        print(f"无效的 JSON 响应: {e}\n {data}")
    except KeyError as e:
        print(f"解析 JSON 错误: {e}\n {data}")

In [ ]:
class TextStorage:
    def __init__(self):
        self.text = ""

    def add_text(self, new_text):
        self.text += str(new_text)

    def get_text(self):
        return self.text
    
    def clear_text(self):
        self.text = ""

In [ ]:
with open("./data/readinglist.json", "r") as f:
    reading_list = json.load(f)

In [ ]:
reading_list,type(reading_list)
T=TextStorage()

In [125]:
prompt = f"""
下面是三个引号括起来的JSON格式数据，增加一个列名为简介的列，把简介补充好,转换成HTML表格,表格要有分割线样式，只回答我html代码:

"""
for book in reading_list: 
    display((json.dumps(book["书名"],ensure_ascii=False)))
    response = get_completion(f"{prompt}+\"\"\"{json.dumps(book)}\"\"\"")
    T.add_text(response)
    

'"人月神话"'

'"操作系统概念"'

'"编译原理"'

'"Unix 编程艺术"'

解析 JSON 错误: 'choices'
 {'error': {'message': 'Rate limit reached for default-gpt-3.5-turbo in organization org-SjJboAKeF2Eoty5Z9L91NDDX on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method.', 'type': 'requests', 'param': None, 'code': None}}


...

'"代码大全"'

'"人工智能：一种现代的方法"'

'"Head First 设计模式"'

'"人类简史：从动物到上帝"'

'"深度学习"'

In [ ]:
HTML(T.get_text())